Reducere la forma Superior Hessenberg
=================================

$\qquad$Oricare ar fi matricea  $A \in \mathbb{R}^{nxn}$ , există o matrice ortogonală  $Q \in \mathbb{R}^{nxn} $ calculabilă printr-o secventă finită de operații aritmetice, astfel încât matricea  $$H = Q^TAQ$$ este superior Hessenberg.
Pentru a obține forma superior Hessenberg este suficient să ¸stim să ”operăm” numai cu reflectori de indice 1. În acest scop introducem următoarele proceduri:

$\qquad\bullet$**Z**(Procedură de calcul a elementelor definitorii  $u$ si $\beta$ ale unui reflector $U_1$ astfel încăt $(U_1x)(2:n) = 0$ și de calcul $x \gets U_1x$, unde $x \in \mathbb{R}^n$ un vector dat.)

1. $\sigma = sign(x_1)\sqrt{\sum\limits_{i=1}^n x_{i}^{2}}$
2. $u_1 = x_1 + \sigma$
3. **pentru** $i = 2:n$
    * $u_i = x_i$
    * $(x_i = 0)$
4. $\beta = u_1\sigma$
5. $x_1 \gets -\sigma$

Sintaxa de apel a procedurii **Z** este  $$[u,\beta,x]=\textbf{Z}(x).$$


$\qquad\bullet$**As**(Proceduă de înmulțire la stânga a unei matrice  $A \in \mathbb{R}^{nxp}$ cu un reflector $U_1$, dat prin elementele sale definitorii $u$ și $\beta$ se calculează $A \gets U_1A$).

1. **pentru** $i = 1:p$
    * $\tau = (\sum\limits_{i=1}^n u_ia_{ij})/\beta$ 
    * **pentru** $i = 1:n$
        + $a_{ij} \gets a_{ij} - \tau u_i$
        
Sintaxa de apel a procedurii **As** este $$A = \textbf{As}(u,\beta,A).$$


$\qquad\bullet$**Ad**(Proceduă de înmulțire la dreapta a unei matrice  $A \in \mathbb{R}^{nxp}$ cu un reflector $U_1$, dat prin elementele sale definitorii $u$ și $\beta$ se calculează $A \gets AU_1$).

1. **pentru** $i = 1:m$
    * $\tau = (\sum\limits_{j=1}^n a_{ij}u_j)/\beta$ 
    * **pentru** $i = 1:n$
        + $a_{ij} \gets a_{ij} - \tau u_j$
        
Sintaxa de apel a procedurii **Ad** este $$A = \textbf{Ad}(u,\beta,A).$$

Utilizând procedurile **Z**, **As** și **Ad**, se obține imediat următorul algoritm.

$\qquad$*ALGORITM(Reducere la forma superior Hessenberg)*((Dată o matrice $A \in \mathbb{R}^{nxn}$ , algoritmul calculează o secvență de reflectori $U_2, U_3, ..., U_{n-1}$ astfel încăt matricea este superior Hessenberg. De asemenea se calculează matricea de transformare $Q = U_2U_3...U_{n-1}$).

1. $Q = I_n$
2. **pentru** $k = 1:n-2$
    * $[u,\beta,A(k + 1:n,k)] = \textbf{Z}(A(k + 1:n,k))$
    * $ A(k + 1:n,k + 1:n) = \textbf{As}(u,\beta,A(k + 1:n,k + 1:n))$
    * $A(1:n,k + 1:n) = \textbf{Ad}(u,\beta,A(1:n,k + 1:n))$
    * $Q(1:n,k + 1:n) = \textbf{Ad}(u,\beta,Q(1:n,k + 1:n))$
    
$\qquad$Pentru apelul algoritmului \textbf{HQ} se utilizează sintaxa generală $$[H,Q] = \textbf{HQ}(A),$$ care exprimă posibilitatea de a memora rezultatele în alte tablouri decât cele inițiale deși calculele se fac cu suprascrierea internă a matricei inițiale.

In [1]:
#functia Ad(inmulteste la dreapta cu un reflector)
import numpy as np
import math
from scipy.linalg import hessenberg

def Ad(A,u,beta):
    n = len(A[0])
    m =len(A)
    for i in range(m):
        suma = 0
        for j in range(n):
            suma = suma + A[i][j]*u[j]
        thau = suma/beta
        for j in range(n):
            A[i][j] = A[i][j] - thau*u[j]
    
    return A

#functia As(inmulteste la stanga cu un reflector)
def As(A,u,beta):
    n = len(A)
    p = len(A[0])
    for j in range(p):
        suma = 0
        for i in range(n):
            suma = suma + u[i]*A[i][j]
        thau = suma/beta
        for i in range(n):
            A[i][j] = A[i][j] - thau*u[i]
            
    return A

#functia Z(calculeaza elementele definitorii reflectorului)
def Z(x):
    n = len(x)
    
    u = np.zeros((n,1))
    suma = 0
    for i in range(n):
        suma = suma + x[i]**2
    xi = np.sign(x[0])*math.sqrt(suma)
    u[0] = x[0] + xi
    for i in range(1,n):
        u[i] = x[i]
        x[i] = 0
    beta = u[0]*xi
    x[0] = -xi
    
    return u, beta, x

#functia HQ(algoritmul de reducere la forma Hessenberg)
def HQ(A):
    n = len(A)
    Q = np.ones((n,n))
    for k in range(n-1):
        u, beta, A[(k+1):n, k]  = Z(A[(k+1):n, k])
        A[(k+1):n, (k+1):n] = As(A[(k+1):n, (k+1):n],u,beta)
        A[:,(k+1):n] = Ad(A[:,(k+1):n],u,beta)
        Q[:,(k+1):n] = Ad(Q[:,(k+1):n],u,beta)
        
    mesaj = 0
    return Q, A, mesaj
#--------------------------------------------------------------------------
n = 4
A = np.random.randn(n,n)
tol = 1e-10
maxiter = 100
#---------------------------------------------------------------------------
B = np.copy(A)
Q, H, mesaj = HQ(B)
if not mesaj:
    Hv,Qv = hessenberg(A,calc_q=True)
    if np.allclose(H.all(),Hv.all()):
        print("Matricea transformata este:")
        print(Hv)
    else:
        print("Matrice calculata gresit")
else:
    print("Eroare HQ")
#---------------------------------------------------------------------------

Matricea transformata este:
[[-1.04736291 -0.14556706  1.14168617  1.66450679]
 [-2.05521449  0.0258447   1.22786192 -1.01289432]
 [ 0.          1.50208151  0.17050476  0.84051174]
 [ 0.          0.          1.24761888 -0.45232602]]
